<a href="https://colab.research.google.com/github/HamidBekamiri/Blog-Posts/blob/main/FuzzyDataMatchingWithGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Title: Fuzzy Data Matching with GPT-based Embeddings and Nearest Neighbors


##High-Level Executive Summary:
Data matching, the process of identifying similar records across multiple datasets, is a common challenge in data management. In this code, we will use a powerful approach to fuzzy data matching using GPT-based embeddings and Nearest Neighbors. We demonstrate how to leverage the OpenAI GPT model to generate embeddings for textual data and utilize the Nearest Neighbors algorithm for finding close matches. By following the provided code and explanations, you'll be equipped with a practical solution to efficiently match fuzzy records across datasets.

In [ ]:
import os
import openai
os.environ['OPENAI_API_KEY'] = 'sk-'
openai.api_key = os.getenv("OPENAI_API_KEY")

In [1]:
import openai
from sklearn.neighbors import NearestNeighbors
import pandas as pd

def get_embedding(text):
    result = openai.Embedding.create(
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]

def fuzzy_match(target_df, source_df, columns, threshold=0.85):
    """
    Function to perform fuzzy matching between two dataframes on specified columns.

    Parameters:
    target_df (pd.DataFrame): The dataframe to be matched to.
    source_df (pd.DataFrame): The dataframe to be matched from.
    columns (list of str): The columns to perform fuzzy matching on.
    threshold (float, optional): The cosine similarity threshold for a match to be considered 'good'. Defaults to 0.85.

    Returns:
    pd.DataFrame: A new dataframe where each specified column in source_df is matched against the corresponding column in target_df,
                  with similarity scores and 'good'/'bad' match indicators for each column.
    """

    matched_results = source_df[columns].copy()

    for column in columns:
        target_df[column + '_embeddings'] = target_df[column].apply(get_embedding)
        source_df[column + '_embeddings'] = source_df[column].apply(get_embedding)

        nn = NearestNeighbors(n_neighbors=1, metric='cosine').fit(target_df[column + '_embeddings'].to_list())
        distances, indices = nn.kneighbors(source_df[column + '_embeddings'].to_list(), return_distance=True)

        matched_results[column + '_matched_to'] = [target_df.loc[indices[i, 0], column] for i in range(source_df.shape[0])]
        matched_results[column + '_similarity'] = 1 - distances
        matched_results[column + '_is_good_match'] = ['good' if 1 - distances[i, 0] >= threshold else 'bad' for i in range(source_df.shape[0])]

    return matched_results


ModuleNotFoundError: ignored

In [2]:
# Assuming you have two dataframes df1 and df2 with 'name' and 'address' fields
df1 = pd.DataFrame({
    'name': ['John Doe', 'Mary Jane', 'Peter Parker'],
    'address': ['123 Main St', '456 Oak St', '789 Pine St'],
    'age': ['10','25','100'],
})

df2 = pd.DataFrame({
    'name': ['Pete Parker', 'Jon Doe', 'John Doe', 'Marry Jane', 'Anita Smith'],
    'address': ['123 Main Street', '457 Oak St', '457', '789 Pine Street', '789'],
    'age': ['10','30','40','90',''],
})

NameError: ignored